# PyTorch
In this exercise, we will look at some basic functionality of PyTorch. Your are free to use other DL frameworks for your exercises and your project. However, the master solutions and code examples will be in PyTorch.

The [PyTorch documentation](https://pytorch.org/docs/stable/index.html) offers information on its functionality. A lot of the time, your specific question will also have been asked on the [PyTorch Forum](https://discuss.pytorch.org/), often with competent answers by the core developers (Google will find the relevant thread for you).

First, we have to install PyTorch. We will install the basic version for this exercise. For your project, if you want to run on a GPU, you'll have to make sure to have a PyTorch version installed that is compatible with the CUDA version of your NVIDIA drivers. PyTorch has an [installation guide](https://pytorch.org/get-started/locally/) that will help you with getting the right version.

In [1]:
# %pip install -U numpy
# %pip install ipywidgets 
# %pip install torch --index-url https://download.pytorch.org/whl/cu117

In [2]:
import torch

torch.cuda.is_available()

True

## Tensor operations
Most of PyTorch's operations have the same name as in NumPy. The basic object for storing data is the `torch.tensor`, the equivalent of the `np.array`. With the help of the [Tensor tutorial](https://pytorch.org/tutorials/beginner/blitz/tensor_tutorial.html), do the following:

- Create a `torch.tensor` with the elements `[[1, 2], [3, 4]]`
- Create a tensor of ones/zeros with the same shape and dtype
- Create a random tensor of the same shape
- Print the tensor's shape, data type and device
- Try to move it to the GPU
- For Mac users: Try to move it to [MPS](https://pytorch.org/docs/stable/notes/mps.html)
- Check out indexing/slicing operations, and how you can assign values to a slice.
- Combine tensors with `torch.cat` and `torch.stack`. What are the differences?
- Multiply tensors, element-wise and with matrix multiplication.

In [3]:
x = torch.Tensor([[1, 2], [3, 4]])

x_ones = torch.ones_like(x, dtype=x.dtype)
x_zeroes = torch.zeros_like(x, dtype=x.dtype)
x_random = torch.rand_like(x, dtype=x.dtype)

print(f"Shape of tensor: {x.shape}")
print(f"Datatype of tensor: {x.dtype}")
print(f"Device tensor is stored on: {x.device}")

x_cuda = x.to("cuda")
print(f"Device of cuda tensor: {x_cuda.device}")

x[:,1] = 0
print(x)

x[1:] = 5
print(x)

# torch.stack: Creates a new dimension while stacking the tensors, the input tensors must have the same shape, returns a new tensor with one more dimension than the input tensors
# torch.cat: concatenates tensors along an existing dimension, the tensors must have the same shape except in the dimension along which they are concatenated, does not add a new dimension.
t1 = torch.cat([x, x, x], dim=1)
print(t1)
t2 = torch.cat([x, x, x], dim=0)
print(t2)

t3 = torch.stack([x, x, x], dim=1)
print(t3)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/home/david/miniconda3/envs/nlp/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/david/miniconda3/envs/nlp/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/david/miniconda3/envs/nlp/lib/python3.9/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/david/miniconda3/envs/nlp/lib/python3.9/site-packages/traitlets/config/application.py", line 1075, in launch_in

Shape of tensor: torch.Size([2, 2])
Datatype of tensor: torch.float32
Device tensor is stored on: cpu
Device of cuda tensor: cuda:0
tensor([[1., 0.],
        [3., 0.]])
tensor([[1., 0.],
        [5., 5.]])
tensor([[1., 0., 1., 0., 1., 0.],
        [5., 5., 5., 5., 5., 5.]])
tensor([[1., 0.],
        [5., 5.],
        [1., 0.],
        [5., 5.],
        [1., 0.],
        [5., 5.]])
tensor([[[1., 0.],
         [1., 0.],
         [1., 0.]],

        [[5., 5.],
         [5., 5.],
         [5., 5.]]])


## Neural Network Basics
Solve the followings tasks with the help of the [Neural networks tutorial](https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html).

The `nn.Module` is the basic class for layers, networks and models. All parameters of an `nn.Module` are automatically discovered by PyTorch and updated by back-propagation.

First, define a neural network (as a subclass of `nn.Module`) with two linear layers and a ReLU non-linearity in between. Make the input, output, and inner dimensions parameters of your network.

In [4]:
import torch.nn as nn
import torch.nn.functional as F

In [5]:
class Net(nn.Module):
  def __init__(self, fc1_in, fc2_in, fc2_out):
    super(Net, self).__init__()

    self.fc1 = nn.Linear(fc1_in, fc2_in)
    self.fc2 = nn.Linear(fc2_in, fc2_out)

  def forward(self, input):
    f1 = F.relu(self.fc1(input))
    out = F.relu(self.fc2(f1))

    return out

In [6]:
model = Net(10, 20, 5)

Move the entire network to the GPU/MPS.

In [7]:
model = model.to("cuda")

Print the parameters of your network.

In [8]:
params = list(model.parameters())
print(len(params))
print(model.parameters)

4
<bound method Module.parameters of Net(
  (fc1): Linear(in_features=10, out_features=20, bias=True)
  (fc2): Linear(in_features=20, out_features=5, bias=True)
)>


Run a single forward-pass with a random input.

In [9]:
input = torch.randn(1, 10).to("cuda")
out = model(input)
print(out)

tensor([[0.0000, 0.4207, 0.0000, 0.1612, 0.0515]], device='cuda:0',
       grad_fn=<ReluBackward0>)


Define a `nn.MSELoss` and a random target.

In [10]:
criterion = nn.MSELoss()
target = torch.randn(5)
target = target.view(1, -1).to("cuda")  # make it the same shape as output

Compute the loss and run backpropagation.

In [11]:
loss = criterion(out, target)
print(loss)

tensor(0.2843, device='cuda:0', grad_fn=<MseLossBackward0>)


Update the parameters of your network with a learning rate of 0.01.

In [12]:
learning_rate = 0.01

model.zero_grad()
loss.backward()

for f in model.parameters():
    f.data.sub_(f.grad.data * learning_rate)

Use the `AdamOptimizer` instead to update your parameters (see the [torch.optim documentation](https://pytorch.org/docs/stable/optim.html)).

In [13]:
import torch.optim as optim

# create your optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# in training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = model(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update